In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train = pd.read_csv(r'/content/drive/MyDrive/train_15.csv', encoding='utf_8_sig')
test = pd.read_csv(r'/content/drive/MyDrive/test_15.csv', encoding='utf_8_sig')


In [4]:
train

,label,text,diff
0,positive,Check types to avoid invalid reads/writes.,diff --git a/src/file.c b/src/file.c\nindex 4d...
1,positive,[CVE-2012-2098] Integrate fallback sort into t...,diff --git a/src/main/java/org/apache/commons/...
2,negative,Merge branch '1.8.x',diff --git a/src/ClientCommand.cpp b/src/Clien...
3,negative,[JBPM-9474] Produce events to Kafka by Signal ...,diff --git a/jbpm-bpmn2/src/main/java/org/jbpm...
4,positive,Resolving Issue #918\n\nMinor XSS and create g...,diff --git a/docs/CHANGELOG b/docs/CHANGELOG\n...
5,positive,CVE-2017-12895/ICMP: Check the availability of...,diff --git a/print-icmp.c b/print-icmp.c\ninde...
6,negative,9.0.4,diff --git a/package.json b/package.json\ninde...
7,negative,"Needs to allow this syscall as well, caught on...",diff --git a/linux_priv.c b/linux_priv.c\ninde...
8,positive,SPOOLSS: Try to avoid an infinite loop.\n\nUse...,diff --git a/epan/dissectors/packet-dcerpc-spo...
9,positive,Update NewslettersController.php\n\nTest if an...,diff --git a/www/guis/users/application/contro...


In [5]:
len(train)

40

In [6]:
train['label'].value_counts()

negative    25
positive    15
Name: label, dtype: int64

In [7]:
test['label'].value_counts()

negative    6322
positive    3750
Name: label, dtype: int64

In [8]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-mpnet-base-v2')

In [9]:
train['embeddings'] = train['text'].apply(lambda x: model.encode(str(x)))
test['embeddings'] = test['text'].apply(lambda x: model.encode(str(x)))

In [10]:
test

,label,text,diff,embeddings
0,positive,Escapes now HTML (JsonBuilder).,diff --git a/src/main/java/org/projectforge/we...,"[-0.04151315, 0.24733387, -0.074037, 0.0535078..."
1,positive,Fix CSV injection issue if server responds wit...,diff --git a/program/plugins/nikto_outdated.pl...,"[-0.009063445, -0.27374032, -0.0040009026, 0.0..."
2,negative,Merge PR #4713: BUILD(installer): Include g15-...,diff --git a/installer/ClientInstaller.cs b/in...,"[-0.18374328, 0.22310455, -0.004842456, -0.082..."
3,negative,Merge pull request #153 from jglick/dependabot...,diff --git a/.github/CODEOWNERS b/.github/CODE...,"[-0.113007486, 0.20437217, -0.013591562, 0.028..."
4,positive,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/txt.c b/coders/txt.c\ninde...,"[-0.10702289, 0.20025887, 0.04798815, 0.008700..."
...,...,...,...,...
10067,positive,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/MagickWand/mogrify.c b/MagickWand...,"[-0.0905226, 0.07581295, 0.045917235, 0.038698..."
10068,positive,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/mat.c b/coders/mat.c\ninde...,"[-0.084771134, 0.12157967, 0.035356797, 0.0241..."
10069,negative,[maven-release-plugin] prepare release promote...,diff --git a/pom.xml b/pom.xml\nindex 92353944...,"[-0.02828617, -0.03947244, -0.056398977, -0.14..."
10070,positive,Taxonomy: Correct the formatting of HTML entit...,diff --git a/wp-admin/js/tags-box.js b/wp-admi...,"[-0.08080343, 0.09955625, -0.045322593, 0.0984..."


In [11]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [12]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [13]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))

Logistic Regression Accuracy: 0.7477164416203336
Logistic Regression Precision: 0.742779743473779
Logistic Regression Recall: 0.7477164416203336
Logistic Regression Recall: 0.7414516343507467
